In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
### Run Time Function
def print_runtime(t1, t2):
    tot_sec = t2.timestamp()-t1.timestamp()
    hours = tot_sec//3600
    minutes = (tot_sec-hours*3600)//60
    seconds = tot_sec-hours*3600-minutes*60

    print("Run Time:", hours, 'hrs', minutes, 'mins', round(seconds), "sec")
    
    return

In [3]:
### Directory of the simulation.py file
simpy_dir = r'C:/abm_runs/rohans'

### Location of Configs
settings = r'C:/abm_runs/rohans/configs/resident/settings_mp.yaml'
configs_resident = r'C:/abm_runs/rohans/configs/resident'
configs_common = r'C:/abm_runs/rohans/configs/common'

### Location of auto_ownership_coefficients
ao_coef_loc = r'C:/abm_runs/rohans/configs/resident'

### Location of AO targets
base_target_loc = r'C:/abm_runs/rohans/calibration/auto_ownership/targets/ao_targets_2021.csv'

### Location of Input and Output data
data = r'C:/abm_runs/rohans/input'
output_dir = r'C:/abm_runs/rohans/output'

In [4]:
### Define dampening or enhancing factor
damp_fac = 1.5

### Read in base targets/frequency
base_share = pd.read_csv(base_target_loc, index_col=0)
base_share

,share
auto_ownership,
0,0.054026
1,0.295868
2,0.398939
3,0.160256
4,0.090911


### Cold Start: 
Run ActivitySim up to auto_ownership_simulate model by editing settings_mp.yaml

In [5]:
i = 0
### Read and backup existing auto_ownership_coefficients
ao_coef = pd.read_csv(ao_coef_loc+'/auto_ownership_coefficients.csv')
ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)

In [5]:
start_time = datetime.datetime.now()

#Run ActivitySim
os.chdir(simpy_dir)
asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir

os.system(asim_command)

0

In [6]:
end_time = datetime.datetime.now()
print_runtime(start_time, end_time)

Run Time: 1.0 hrs 38.0 mins 56 sec


In [8]:
### Read households files
hh_orig = pd.read_csv(output_dir + "/final_households.csv")
hh_orig['weights'] = 1/hh_orig['sample_rate']

### Remove group quarter (gq) households
hh = hh_orig[hh_orig['HHT'].isin([1, 2, 3, 4, 5, 6, 7])]

### Calculate asim shares
asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
asim_share = asim_freq/asim_freq.sum()
# asim_share.reset_index(inplace=True)

### Calculate calibration constants
ao_calib = base_share.rename({'share': 'census'}, axis=1)
ao_calib['model'] = asim_share['weights']
ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
ao_calib['constants'] = np.log(ao_calib['ratio'])

ao_calib.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/calibration run results/ao_calib_summary_'+str(i)+'.csv')
ao_calib

,census,model,ratio,constants
auto_ownership,,,,
0,0.054026,0.063727,0.847767,-0.165149
1,0.295868,0.314506,0.940739,-0.061090
2,0.398939,0.390553,1.021473,0.021246
3,0.160256,0.144961,1.105508,0.100305
4,0.090911,0.086253,1.054008,0.052600


In [23]:
# i=1
# ### Update auto ownership constants
# cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
# ao_coef.loc[97:, 'value'] = np.array(ao_coef.loc[97:, 'value']) + cali_fac.values

# ### Save auto ownership constants
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

### Warm Start
Edit settings.yaml to resume after and go to write tables

In [9]:
for i in range(1, 2):
    print('Iteration -', i)
    
    ### Update auto ownership constants
    cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
    ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

    ### Save auto ownership constants
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

    ### Run ActivitySim
    start_time = datetime.datetime.now()
    
    os.chdir(simpy_dir)
    asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
    os.system(asim_command)

    end_time = datetime.datetime.now()
    print_runtime(start_time, end_time)
    
    ### Read households files
    hh_orig = pd.read_csv(output_dir + "/final_households.csv")
    hh_orig['weights'] = 1/hh_orig['sample_rate']

    ### Remove group quarter (gq) households
    hh = hh_orig[hh_orig['HHT'].isin([1, 2, 3, 4, 5, 6, 7])]

    ### Calculate asim shares
    asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
    asim_share = asim_freq/asim_freq.sum()
    asim_share.reset_index(inplace=True)

    ### Calculate calibration constants
    ao_calib = base_share.rename({'share': 'census'}, axis=1)
    ao_calib['model'] = asim_share['weights']
    ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
    ao_calib['constants'] = np.log(ao_calib['ratio'])

    ao_calib.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/calibration run results/ao_calib_summary_'+str(i)+'.csv')
    print(ao_calib, '\n')

Iteration - 1
Run Time: 0.0 hrs 40.0 mins 27 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.052817  1.022892   0.022633
1               0.295868  0.315346  0.938232  -0.063758
2               0.398939  0.387497  1.029529   0.029101
3               0.160256  0.157842  1.015295   0.015179
4               0.090911  0.086498  1.051013   0.049755 



In [11]:
for i in range(2, 4):
    print('Iteration -', i)
    
    ### Update auto ownership constants
    cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
    ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

    ### Save auto ownership constants
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

    ### Run ActivitySim
    start_time = datetime.datetime.now()
    
    os.chdir(simpy_dir)
    asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
    os.system(asim_command)

    end_time = datetime.datetime.now()
    print_runtime(start_time, end_time)
    
    ### Read households files
    hh_orig = pd.read_csv(output_dir + "/final_households.csv")
    hh_orig['weights'] = 1/hh_orig['sample_rate']

    ### Remove group quarter (gq) households
    hh = hh_orig[hh_orig['HHT'].isin([1, 2, 3, 4, 5, 6, 7])]

    ### Calculate asim shares
    asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
    asim_share = asim_freq/asim_freq.sum()
    asim_share.reset_index(inplace=True)

    ### Calculate calibration constants
    ao_calib = base_share.rename({'share': 'census'}, axis=1)
    ao_calib['model'] = asim_share['weights']
    ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
    ao_calib['constants'] = np.log(ao_calib['ratio'])

    ao_calib.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/calibration run results/ao_calib_summary_'+str(i)+'.csv')
    print(ao_calib, '\n')

Iteration - 2
Run Time: 0.0 hrs 37.0 mins 42 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.053712  1.005844   0.005827
1               0.295868  0.309638  0.955529  -0.045490
2               0.398939  0.391672  1.018555   0.018385
3               0.160256  0.155571  1.030115   0.029671
4               0.090911  0.089408  1.016815   0.016675 

Iteration - 3
Run Time: 0.0 hrs 32.0 mins 48 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.053712  1.005844   0.005827
1               0.295868  0.306149  0.966418  -0.034158
2               0.398939  0.393179  1.014649   0.014543
3               0.160256  0.157956  1.014564   0.014459
4               0.090911  0.089004  1.021425   0.021199 



### Testing

In [ ]:
# i = 1
# print('Iteration -', i)
    
# ### Update auto ownership constants
# cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
# ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

In [ ]:
# ### Save auto ownership constants
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

In [13]:
# ### Run ActivitySim
# start_time = datetime.datetime.now()

# os.chdir(simpy_dir)
# asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
# os.system(asim_command)

# end_time = datetime.datetime.now()
# print_runtime(start_time, end_time)

Run Time: 0.0 hrs 32.0 mins 25 sec


In [14]:
# ### Read households files
# hh = pd.read_csv(output_dir + "/final_households.csv")
# hh['weights'] = 1/hh['sample_rate']

# ### Calculate asim shares
# asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
# asim_share = asim_freq/asim_freq.sum()
# asim_share.reset_index(inplace=True)
# asim_share

,auto_ownership,weights
0,1,0.398707
1,2,0.372920
2,3,0.147883
3,4,0.080490


In [ ]:
# ### Calculate calibration constants
# ao_calib = base_share.rename({'share': 'census'}, axis=1)
# ao_calib['model'] = asim_share['weights']
# ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
# ao_calib['constants'] = np.log(ao_calib['ratio'])

# ao_calib.to_csv('C:/abm_runs/rohans/calibration/auto_ownership/calibration run results/ao_calib_summary_'+str(i)+'.csv')
# print(ao_calib, '\n')